In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import re
import importlib

module_path = Path(os.path.abspath("")).parent.parent
print(module_path)
sys.path.append(str(module_path))

from notebooks.inference import utils, rule_based_review_split

importlib.reload(utils)


processed_folder_path = f'{module_path}/data/processed'

/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/peerq-generation
/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/peerq-generation


In [2]:
all_reviews = []


# Step 1: Check if the processed folder exists
files = os.listdir(processed_folder_path)

# Step 4: Iterate over the files and process them
for file in files:
    file_path = os.path.join(processed_folder_path, file)
    if os.path.isfile(file_path) and file.endswith('.csv'):
        print(f'Processing file: {file}')
        df = pd.read_csv(file_path)
        print(f'Number of reviews from this source: {len(df)}')

        for index, row in df.iterrows():
            review = row['focused_review']

            if 'acl' in file:
                source = row['filename'].split('josn')[0]
            elif 'ARR' in file:
                source = row['paped_id']
            elif 'nips'in file or 'iclr' in file:
                source = row['review_id']
            
            all_reviews.append({'source': source, 'focused_review': review})

    
all_reviews_df = pd.DataFrame(all_reviews)
all_reviews_df['focused_review'] = all_reviews_df['focused_review'].astype(str)






split_review_column = []
## Split the reviews into points
for i,r in tqdm(all_reviews_df.iterrows(),total=len(all_reviews_df)):
    focused_review = re.sub(r'\s+', ' ', r['focused_review'])

    ## remove the first occurence of weakness or Weakness
    remove_list = ['weakness', 'Weakness','weaknesses', 'Weaknesses']
    for word in remove_list:
        if focused_review.startswith(word) or focused_review.startswith(word + ':'):
            focused_review  = ' '.join(focused_review.split()[1:])
            
    # Split thre review, and merge short sentences
    splitted_reviews = rule_based_review_split.split_into_points_new(focused_review)
    splitted_reviews = utils.merge_short_sentences(splitted_reviews)

    ### TODO: Come up with a better way to merge the reviews, Why can't we save them as list
    # all_reviews_filtered.at[i, 'split_review'] = '$$$'.join(splitted_reviews)

    ##########333 Filter short reviews  ###################
    final_split_review = []
    for review in splitted_reviews:
        if len(review.split()) >= 10: 
            final_split_review.append(review)


    # print(splitted_reviews)
    # print(final_split_review)

    split_review_column.append(final_split_review)
    # all_reviews_df['split_review'].iloc[i] = final_split_review
    # all_reviews_df.at[i, 'split_review'] = ['dsfds','fdfdfd']



all_reviews_df['split_review'] = split_review_column
### Filter short reviews
all_reviews_filtered = utils.filter_reviews(all_reviews_df, 'split_review')
# print(f'Number of reviews before and after filtering: {len(all_reviews_df)} and {len(all_reviews_filtered)}')

all_reviews_filtered.to_csv(f'{module_path}/data/all_reviews.csv', index=False)

Processing file: iclr_reviews.csv


/tmp/ipykernel_1506884/2295442157.py:12: DtypeWarning: Columns (1,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Number of reviews from this source: 43198
Processing file: reviewer2_ARR_2022_reviews.csv
Number of reviews from this source: 573
Processing file: acl2017_reviews.csv
Number of reviews from this source: 205
Processing file: nips_reviews.csv


/tmp/ipykernel_1506884/2295442157.py:12: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Number of reviews from this source: 39684


100%|██████████| 83660/83660 [00:35<00:00, 2384.54it/s]


filtering reviwes, and only considering the ones with length of one STD away form mean.
Number of reviews before filtering: 83660
Number of reviews before removing one-point reviews: 46112
Number of reviews with more than one point: 6580
Number of the review points before filtering: 23866
mean: 80.07755803234727 std: 105.95737958465777 min: -25.879821552310503 max: 186.03493761700503


100%|██████████| 6580/6580 [00:00<00:00, 12922.21it/s]


Number of the reviews after filtering: 23866


In [3]:
import ast


all  = pd.read_csv(f'{module_path}/data/all_reviews.csv')

num_of_points = []
no_points = 0

sources_cnt = {}
sources_points_stats = {}
for i,r in all.iterrows():
    points = ast.literal_eval(r['split_review'])
    src = r['source'].split('_')[0]
    if len(points) == 1:
        no_points += 1
    else:
        sources_points_stats[src] = sources_points_stats[src] + len(points) if src in sources_points_stats else len(points)
    sources_cnt[src] = sources_cnt[src] + 1 if src in sources_cnt else 1
    num_of_points.append(len(points))


print(f'Number of reviews: {len(all)}')
print(f'Number of reviews with one point: {no_points}')
print(f'Number of reviews with more than one point: {len(all) - no_points}')
print(f'Precentage of reviews with one point: {no_points/len(all)}')
print(f'total number of points: {sum(num_of_points)}')
print(f'Average number of points: {sum(num_of_points)/len(num_of_points)}')
print(f'Max number of points: {max(num_of_points)}')
print(f'Min number of points: {min(num_of_points)}')

    


print(f'Number of samples of each source with more that one point')
for source, count in sources_cnt.items():
    print(f'{source}: {count} ')

print(f'Number of points for each source')
for source, count in sources_points_stats.items():
    print(f'{source}: {count}')


Number of reviews: 6580
Number of reviews with one point: 0
Number of reviews with more than one point: 6580
Precentage of reviews with one point: 0.0
total number of points: 23866
Average number of points: 3.6270516717325227
Max number of points: 23
Min number of points: 2
Number of samples of each source with more that one point
ICLR: 2274 
ARR: 406 
ACL: 169 
NIPS: 3731 
Number of points for each source
ICLR: 7820
ARR: 2017
ACL: 756
NIPS: 13273


In [4]:
with open('/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/peerq-generation/outputs/test_new_spliting.txt','w')as f:


    for i,r in all.sample(500).iterrows():
        focused_review = r['focused_review']
        splitted_reviews = ast.literal_eval(r['split_review'])
        f.write(f'Focused review:\n\n{r["focused_review"]}\n\n')
        for sr in splitted_reviews:
            f.write(f'Review Point: {sr}\n')
        f.write('='*50 + '\n\n')